In [1]:
import numpy as np
import pickle
import pandas as pd
import json
import math
import torch
import string
from random import randint
from collections import defaultdict, Counter
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torchvision

import torch.nn.functional as F
import torch.nn as nn
import os
from torchvision import utils
from itertools import chain
import torch.utils.data as data_utils

import random
from tqdm import tqdm
from inception_score import get_inception_score

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
print(torch.__version__)


1.10.2


In [4]:
# class Generator(nn.Module):
#     def __init__(self):
#         super(Generator, self).__init__()
#         self.conv1=nn.ConvTranspose2d(in_channels=100, out_channels=1024, kernel_size=4, stride=1, padding=0)
#         self.batch1=nn.BatchNorm2d(num_features=1024)
#         self.activ1=nn.ReLU()
#         # State (1024x4x4)
#         self.conv2=nn.ConvTranspose2d(in_channels=1024, out_channels=512, kernel_size=4, stride=2, padding=1)
#         self.batch2=nn.BatchNorm2d(num_features=512)
#         self.activ2=nn.ReLU()
#         # State (512x8x8)
#         self.conv3=nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=4, stride=2, padding=1)
#         self.batch3=nn.BatchNorm2d(num_features=256)
#         self.activ3=nn.ReLU()
#         # State (256x16x16)
#         self.conv4=nn.ConvTranspose2d(in_channels=256, out_channels=3, kernel_size=4, stride=2, padding=1)
#         self.output=nn.Tanh()
#     def forward(self,x):
#         x=self.conv1(x)
#         x=self.batch1(x)
#         x=self.activ1(x)
#         x=self.conv2(x)
#         x=self.batch2(x)
#         x=self.activ2(x)     
#         x=self.conv3(x)
#         x=self.batch3(x)
#         x=self.activ3(x)
#         x=self.conv4(x)
#         x=self.output(x)
#         return x

In [5]:
# class Generator(torch.nn.Module):
#     def __init__(self, channels=3):
#         super().__init__()
#         # Filters [1024, 512, 256]
#         # Input_dim = 100
#         # Output_dim = C (number of channels)
#         self.main_module = nn.Sequential(
#             # Z latent vector 100
#             nn.ConvTranspose2d(in_channels=100, out_channels=1024, kernel_size=4, stride=1, padding=0),
#             nn.BatchNorm2d(num_features=1024),
#             nn.ReLU(True),

#             # State (1024x4x4)
#             nn.ConvTranspose2d(in_channels=1024, out_channels=512, kernel_size=4, stride=2, padding=1),
#             nn.BatchNorm2d(num_features=512),
#             nn.ReLU(True),

#             # State (512x8x8)
#             nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=4, stride=2, padding=1),
#             nn.BatchNorm2d(num_features=256),
#             nn.ReLU(True),

#             # State (256x16x16)
#             nn.ConvTranspose2d(in_channels=256, out_channels=channels, kernel_size=4, stride=2, padding=1))
#             # output of main module --> Image (Cx32x32)

#         self.output = nn.Tanh()

#     def forward(self, x):
#         x = self.main_module(x)
#         return self.output(x)

In [6]:
img_shape = (3, 32, 32)

In [7]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(100, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.shape[0], *img_shape)
        return img


In [8]:
# class Discriminator(nn.Module):
#     def __init__(self):
#         super(Discriminator, self).__init__()
#         self.conv1=nn.Conv2d(in_channels=3, out_channels=256, kernel_size=4, stride=2, padding=1)
#         self.activ1=nn.LeakyReLU(0.2, inplace=True)
#         # State (1024x4x4)
#         self.conv2=nn.Conv2d(in_channels=256, out_channels=512, kernel_size=4, stride=2, padding=1)
#         self.batch2=nn.BatchNorm2d(512)
#         self.activ2=nn.LeakyReLU(0.2, inplace=True)
#         self.conv3=nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=4, stride=2, padding=1)
#         self.batch3=nn.BatchNorm2d(1024)
#         self.activ3=nn.LeakyReLU(0.2, inplace=True)
#         # State (1024x4x4)

#         self.conv4=nn.Conv2d(in_channels=1024, out_channels=1, kernel_size=4, stride=1, padding=0)
#     def forward(self,x):
#         x=self.conv1(x)
#         x=self.activ1(x)
#         x=self.conv2(x)
#         x=self.batch2(x)
#         x=self.activ2(x)     
#         x=self.conv3(x)
#         x=self.batch3(x)
#         x=self.activ3(x)
#         x=self.conv4(x)
#         return x

In [9]:
# class Discriminator(torch.nn.Module):
#     def __init__(self, channels=3):
#         super().__init__()
#         # Filters [256, 512, 1024]
#         # Input_dim = channels (Cx64x64)
#         # Output_dim = 1
#         self.main_module = nn.Sequential(
#             # Image (Cx32x32)
#             nn.Conv2d(in_channels=channels, out_channels=256, kernel_size=4, stride=2, padding=1),
#             nn.BatchNorm2d(num_features=256),
#             nn.LeakyReLU(0.2, inplace=True),

#             # State (256x16x16)
#             nn.Conv2d(in_channels=256, out_channels=512, kernel_size=4, stride=2, padding=1),
#             nn.BatchNorm2d(num_features=512),
#             nn.LeakyReLU(0.2, inplace=True),

#             # State (512x8x8)
#             nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=4, stride=2, padding=1),
#             nn.BatchNorm2d(num_features=1024),
#             nn.LeakyReLU(0.2, inplace=True))
#             # output of main module --> State (1024x4x4)

#         self.output = nn.Sequential(
#             # The output of D is no longer a probability, we do not apply sigmoid at the output of D.
#             nn.Conv2d(in_channels=1024, out_channels=1, kernel_size=4, stride=1, padding=0))


#     def forward(self, x):
#         x = self.main_module(x)
#         return self.output(x)

In [10]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
        )

    def forward(self, img):
        img_flat = img.view(img.shape[0], -1)
        validity = self.model(img_flat)
        return validity

In [11]:
disc_model=Discriminator()
gen_model=Generator()
disc_model.to(device)
gen_model.to(device)


Generator(
  (model): Sequential(
    (0): Linear(in_features=100, out_features=128, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Linear(in_features=128, out_features=256, bias=True)
    (3): BatchNorm1d(256, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Linear(in_features=256, out_features=512, bias=True)
    (6): BatchNorm1d(512, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Linear(in_features=512, out_features=1024, bias=True)
    (9): BatchNorm1d(1024, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Linear(in_features=1024, out_features=3072, bias=True)
    (12): Tanh()
  )
)

In [12]:
gen_model

Generator(
  (model): Sequential(
    (0): Linear(in_features=100, out_features=128, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Linear(in_features=128, out_features=256, bias=True)
    (3): BatchNorm1d(256, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Linear(in_features=256, out_features=512, bias=True)
    (6): BatchNorm1d(512, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Linear(in_features=512, out_features=1024, bias=True)
    (9): BatchNorm1d(1024, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Linear(in_features=1024, out_features=3072, bias=True)
    (12): Tanh()
  )
)

In [13]:
disc_model

Discriminator(
  (model): Sequential(
    (0): Linear(in_features=3072, out_features=512, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): LeakyReLU(negative_slope=0.2, inplace=True)
    (4): Linear(in_features=256, out_features=1, bias=True)
  )
)

In [14]:
def create_dataset(batchsize):
    trans = transforms.Compose([
            transforms.Resize(32),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ])

    train_dataset = torchvision.datasets.CIFAR10(root='./files/', train=True, download=True, transform=trans)
    test_dataset = torchvision.datasets.CIFAR10(root='./files/', train=False, download=True, transform=trans)

    
    train_dataloader = data_utils.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataloader = data_utils.DataLoader(test_dataset,  batch_size=batch_size, shuffle=True)
    return train_dataloader,test_dataloader

In [15]:
learn_rate=0.00005
epochs=200
batch_size=64
critic_iter = 5
generator_iters=15000
weight_cliping_limit=0.01
beta_1,beta_2=0.5,0.999

In [16]:
gen_optim=torch.optim.RMSprop(gen_model.parameters(), lr=learn_rate)
disc_optim=torch.optim.RMSprop(disc_model.parameters(), lr=learn_rate)

In [17]:
def save_model(gen,disc):
    torch.save(gen.state_dict(), './generator_wgan.pkl')
    torch.save(disc.state_dict(), './discriminator_wgan.pkl')
    print('Models save to ./generator_wgan.pkl & ./discriminator_wgan.pkl ')

In [18]:
SAVE_RATE = 1000


In [19]:
# def train(train_loader):
#     file = open("inception_score_graph_wgan.txt", "w")

#     # Now batches are callable self.data.next()
#     data = collate_batches(train_loader)

#     one = torch.FloatTensor(1, dtype=torch.float).to(device)
#     mone = (one * -1).to(device)
    
#     for g_iter in tqdm(range(total_iterations)):

#         # Requires grad, Generator requires_grad = False
#         for para in disc_model.parameters():
#             para.requires_grad = True

#         # Train Dicriminator forward-loss-backward-update self.critic_iter times while 1 Generator forward-loss-backward-update
#         for d_iter in range(critic_iter):
#             disc_model.zero_grad()

#             # Clamp parameters to a range [-c, c], c=self.weight_cliping_limit
#             for para in disc_model.parameters():
#                 para.data.clamp_(-weight_cliping_limit, weight_cliping_limit)

#             images = data.__next__()
#             # Check for batch to have full batch_size
#             if (images.size()[0] != batch_size):
#                 continue

#             z = torch.rand((batch_size, 100, 1, 1)).to(device)
#             images=images.to(device)


#             # Train discriminator
#             # WGAN - Training discriminator more iterations than generator
#             # Train with real images
#             d_loss_real = disc_model(images)
#             d_loss_real = d_loss_real.mean(0).view(1)
#             d_loss_real.backward(one)

#             # Train with fake images
#             z = torch.randn(batch_size, 100, 1, 1).to(device)
#             fake_images = gen_model(z)
#             d_loss_fake = disc_model(fake_images)
#             d_loss_fake = d_loss_fake.mean(0).view(1)
#             d_loss_fake.backward(mone)

#             d_loss = d_loss_fake - d_loss_real
#             Wasserstein_D = d_loss_real - d_loss_fake
#             disc_optim.step()
# #             print(f'  Discriminator iteration: {d_iter}/{critic_iter}, loss_fake: {d_loss_fake.data}, loss_real: {d_loss_real.data}')



#         # Generator update
#         for para in disc_model.parameters():
#             para.requires_grad = False  # to avoid computation

#         gen_model.zero_grad()

#         # Train generator
#         # Compute loss with fake images
#         z = torch.randn(batch_size, 100, 1, 1).to(device)
#         fake_images =  gen_model(z)
#         g_loss = disc_model(fake_images)
#         g_loss = g_loss.mean().mean(0).view(1)
#         g_loss.backward(one)
#         g_cost = -g_loss
#         gen_optim.step()
# #         print(f'Generator iteration: {g_iter}/{total_iterations}, g_loss: {g_loss.data}')
#         if (g_iter) % SAVE_RATE == 0:
#             # Workaround because graphic card memory can't store more than 830 examples in memory for generating image
#             # Therefore doing loop and generating 800 examples and stacking into list of samples to get 8000 generated images
#             # This way Inception score is more correct since there are different generated examples from every class of Inception model
#             sample_list = []
#             for i in range(10):
#                 z = Variable(torch.randn(800, 100, 1, 1)).to(device)
#                 samples = gen_model(z)
#                 sample_list.append(samples.data.cpu().numpy())
            
#             # Flattening list of list into one list
#             new_sample_list = list(chain.from_iterable(sample_list))
# #             print("Calculating Inception Score over 8k generated images")
#             # Feeding list of numpy arrays
#             inception_score = get_inception_score(new_sample_list, cuda=True, batch_size=32,
#                                                   resize=True, splits=10)

#             if not os.path.exists('training_result_images_wgan/'):
#                 os.makedirs('training_result_images_wgan/')

#             # Denormalize images and save them in grid 8x8
#             z = Variable(torch.randn(800, 100, 1, 1)).to(device)
#             samples = gen_model(z)
#             samples = samples.mul(0.5).add(0.5)
#             samples = samples.data.cpu()[:64]
#             grid = utils.make_grid(samples)
#             utils.save_image(grid, 'training_result_images_wgan/img_generatori_iter_{}_wgan.png'.format(str(g_iter).zfill(3)))



In [20]:
# def train(train_loader):
#         file = open("inception_score_graph_wgan.txt", "w")

#         # Now batches are callable self.data.next()
#         data = collate_batches(train_loader)

#         one = torch.FloatTensor([1]).to(device)
#         mone = (one * -1).to(device)

#         for g_iter in tqdm(range(generator_iters)):

#             # Requires grad, Generator requires_grad = False
#             for para in disc_model.parameters():
#                 para.requires_grad = True

#             # Train Dicriminator forward-loss-backward-update self.critic_iter times while 1 Generator forward-loss-backward-update
#             for d_iter in range(critic_iter):
#                 disc_model.zero_grad()

#                 # Clamp parameters to a range [-c, c], c=self.weight_cliping_limit
#                 for para in disc_model.parameters():
#                     para.data.clamp_(-weight_cliping_limit, weight_cliping_limit)

#                 images = data.__next__()
#                 images=images.to(device)
#                 # Check for batch to have full batch_size
#                 if (images.size()[0] != batch_size):
#                     continue

#                 z = Variable(torch.randn((batch_size, 100, 1, 1))).to(device)



#                 # Train discriminator
#                 # WGAN - Training discriminator more iterations than generator
#                 # Train with real images
#                 d_loss_real = disc_model(images)
#                 d_loss_real = d_loss_real.mean(0).view(1)
#                 d_loss_real.backward(one)

#                 # Train with fake images
#                 z = Variable(torch.randn((batch_size, 100, 1, 1))).to(device)
#                 fake_images = gen_model(z)
#                 d_loss_fake = disc_model(fake_images)
#                 d_loss_fake = d_loss_fake.mean(0).view(1)
#                 d_loss_fake.backward(mone)

#                 d_loss = d_loss_fake - d_loss_real
#                 Wasserstein_D = d_loss_real - d_loss_fake
#                 disc_optim.step()
# #                 print(f'  Discriminator iteration: {d_iter}/{critic_iter}, loss_fake: {d_loss_fake.data}, loss_real: {d_loss_real.data}')



#             # Generator update
#             for para in disc_model.parameters():
#                 para.requires_grad = False  # to avoid computation

#             gen_model.zero_grad()

#             # Train generator
#             # Compute loss with fake images
#             z = Variable(torch.randn(batch_size, 100, 1, 1)).to(device)
#             fake_images = gen_model(z)
#             g_loss = disc_model(fake_images)
#             g_loss = g_loss.mean().mean(0).view(1)
#             g_loss.backward(one)
#             g_cost = -g_loss
#             gen_optim.step()
# #             print(f'Generator iteration: {g_iter}/{self.generator_iters}, g_loss: {g_loss.data}')

#             # Saving model and sampling images every 1000th generator iterations
#             if (g_iter) % SAVE_RATE == 0:
#                 # Workaround because graphic card memory can't store more than 830 examples in memory for generating image
#                 # Therefore doing loop and generating 800 examples and stacking into list of samples to get 8000 generated images
#                 # This way Inception score is more correct since there are different generated examples from every class of Inception model
#                 sample_list = []
#                 for i in range(10):
#                     z = Variable(torch.randn(800, 100, 1, 1)).to(device)
#                     samples = gen_model(z)
#                     sample_list.append(samples.data.cpu().numpy())
                
#                 # Flattening list of list into one list
#                 new_sample_list = list(chain.from_iterable(sample_list))
# #                 print("Calculating Inception Score over 8k generated images")
#                 # Feeding list of numpy arrays
#                 inception_score = get_inception_score(new_sample_list, cuda=True, batch_size=32,
#                                                       resize=True, splits=10)

#                 if not os.path.exists('training_result_images_wgan/'):
#                     os.makedirs('training_result_images_wgan/')

#                 # Denormalize images and save them in grid 8x8
#                 z = Variable(torch.randn(800, 100, 1, 1)).to(device)
#                 samples = gen_model(z)
#                 samples = samples.mul(0.5).add(0.5)
#                 samples = samples.data.cpu()[:64]
#                 grid = utils.make_grid(samples)
#                 utils.save_image(grid, 'training_result_images_wgan/img_generatori_iter_{}.png'.format(str(g_iter).zfill(3)))

#                 # Testing
                
#         file.close()

#         # Save the trained parameters
#         save_model(gen_model,disc_model)


In [21]:
Tensor = torch.cuda.FloatTensor

In [22]:
def train(train_loader):
    file = open("inception_score_graph_wgan.txt", "w")
    generator_iter = 0

    for epoch in tqdm(range(epochs)):

        for i, (imgs, _) in enumerate(train_loader):

            # Configure input
            real_imgs = Variable(imgs.type(Tensor))

            # ---------------------
            #  Train Discriminator
            # ---------------------

            disc_optim.zero_grad()

            # Sample noise as generator input
            z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], 100))))

            # Generate a batch of images
            fake_imgs = gen_model(z).detach()
            # Adversarial loss
            loss_disc = -torch.mean(disc_model(real_imgs)) + torch.mean(disc_model(fake_imgs))


            loss_disc.backward()
            disc_optim.step()

            # Clip weights of discriminator
            for p in disc_model.parameters():
                p.data.clamp_(-weight_cliping_limit, weight_cliping_limit)

            # Train the generator every n_critic iterations
            if i % critic_iter == 0:

                # -----------------
                #  Train Generator
                # -----------------

                gen_optim.zero_grad()

                # Generate a batch of images
                gen_imgs = gen_model(z)
                # Adversarial loss
                loss_gen = -torch.mean(disc_model(gen_imgs))

                loss_gen.backward()
                gen_optim.step()


            generator_iter+=1

            if generator_iter % 1000 == 0:
                # Workaround because graphic card memory can't store more than 830 examples in memory for generating image
                # Therefore doing loop and generating 800 examples and stacking into list of samples to get 8000 generated images
                # This way Inception score is more correct since there are different generated examples from every class of Inception model
                sample_list = []
                for i in range(10):
                    z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], 100))))
                    samples = gen_model(z)
                    sample_list.append(samples.data.cpu().numpy())

                # Flattening list of list into one list
                new_sample_list = list(chain.from_iterable(sample_list))
        #                 print("Calculating Inception Score over 8k generated images")
                # Feeding list of numpy arrays
                inception_score = get_inception_score(new_sample_list, cuda=True, batch_size=32,
                                                      resize=True, splits=10)

                if not os.path.exists('training_result_images_wgan/'):
                    os.makedirs('training_result_images_wgan/')

                # Denormalize images and save them in grid 8x8
                z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], 100))))
                samples = gen_model(z)
                samples = samples.mul(0.5).add(0.5)
                samples = samples.data.cpu()[:64]
                grid = utils.make_grid(samples)
                utils.save_image(grid, 'training_result_images_wgan/img_generatori_iter_{}.png'.format(str(generator_iter).zfill(3)))
                output = str(generator_iter) + " " + str(inception_score[0]) + "\n"
                file.write(output)
                # Testing

    file.close()

            # Save the trained parameters
    save_model(gen_model,disc_model)


In [23]:
def collate_batches( data_loader):
        while True:
            for i, (images, _) in enumerate(data_loader):
                yield images

In [24]:
def generate_img(noise, number_of_images):
    samples = gen(z).data.cpu().numpy()[:number_of_images]
    generated_images = []
    for sample in samples:
        generated_images.append(sample.reshape(3, 32, 32))
    return generated_images

In [25]:
def load_model(D_model_filename, G_model_filename):
    D_model_path = os.path.join(os.getcwd(), D_model_filename)
    G_model_path = os.path.join(os.getcwd(), G_model_filename)
    disc_model.load_state_dict(torch.load(D_model_path))
    gen_model.load_state_dict(torch.load(G_model_path))
    print('Generator model loaded from {}.'.format(G_model_path))
    print('Discriminator model loaded from {}-'.format(D_model_path))

In [26]:
epochs

200

In [27]:
train_loader,_=create_dataset(batch_size)
train(train_loader)
save_model(gen_model,disc_model)


Files already downloaded and verified
Files already downloaded and verified


  0%|          | 1/200 [00:08<28:32,  8.60s/it]/home/nkulshr/.conda/envs/DL/lib/python3.9/site-packages/torch/nn/functional.py:3631: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(
/home/nkulshr/DL_HW3/inception_score.py:43: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x).data.cpu().numpy()
100%|██████████| 200/200 [31:02<00:00,  9.31s/it]


Models save to ./generator_wgan.pkl & ./discriminator_wgan.pkl 
Models save to ./generator_wgan.pkl & ./discriminator_wgan.pkl 


In [28]:
def evaluate(test_loader, D_model_path='./discriminator_dcgan.pkl', G_model_path='./generator_dcgan.pkl'):
#     load_model( G_model_path,D_model_path)
    z = Variable(Tensor(np.random.normal(0, 1, (batch_size, 100))))
    samples = gen_model(z)
    samples = samples.mul(0.5).add(0.5)
    samples = samples.data.cpu()
    grid = utils.make_grid(samples)
    print("Grid of 8x8 images saved to 'wgan_model_image.png'.")
    utils.save_image(grid, 'wgan_model_image.png')

In [29]:
_,test_loader=create_dataset(batch_size)
evaluate(test_loader)

Files already downloaded and verified
Files already downloaded and verified


RuntimeError: mat1 and mat2 shapes cannot be multiplied (6400x1 and 100x128)